In [1]:
# Import dependencies

import numpy as np 
from citipy import citipy
import gmaps
import requests
import pandas as pd 
from datetime import datetime

from config import weather_api_key
from config import g_key

In [2]:
# Generate 2,000 random latitudes and longitudes, and zip them as a list of tuples
lats = np.random.uniform(-90.0,90.0, size=1500)
lngs = np.random.uniform(-180.0,180.0,size=1500)
lats_lngs = zip(lats, lngs)
coordinates = list(lats_lngs)

In [3]:
# Get the nearest city using citipy module
cities = list()
# Identify the nearest city for each latitude and longitude combination
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0],coordinate[1]).city_name
    
    # If the city is unique, then add to the cities list
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient control
print(cities[:10], 'Generate', len(cities))

['tumannyy', 'makat', 'avarua', 'norman wells', 'barrow', 'pasighat', 'georgetown', 'magugu', 'yar-sale', 'saint-georges'] Generate 628


In [ ]:
# Perform an API call with the OpenWeatherMap.

# initial counters for log and sets
record_count = 1 
set_count = 1

city_data = list()
basic_url = "http://api.openweathermap.org/data/2.5/weather?units=imperial&APPID=" + weather_api_key

print('Beginning Data Retrieval     ')
print("-----------------------------")
# use enumerate() method to loop index and item simutanously 
for i, item in enumerate(cities):
    if i % 50 == 0 and i != 0 :
        record_count = 1  
        set_count +=1 
    # build URL for API call
    url = basic_url + '&q=' + item
    # Log the URL, record, and set numbers and the city.
    print(f'Processing Record {record_count} of Set {set_count} | {item}')
    # increment record count
    record_count +=1

    try:
        js = requests.get(url).json()
        city_name = item
        city_country = js['sys']['country']
        city_UTCdate = js['dt']
        city_date = datetime.utcfromtimestamp(city_UTCdate).strftime('%Y-%m-%d %H:%M:%S')
        city_lat = js['coord']['lat']
        city_lon = js['coord']['lon']
        city_max_temp = js['main']['temp_max']
        city_humidity = js['main']['humidity']
        city_cload = js['clouds']['all']
        city_wind_speed = js['wind']['speed']
        city_weather_description = js['weather'][0]['description']

        # raining info
        try:
            js['weather'][0]['main'] == 'Rain'
            city_rain = js['rain']['3h']
        except:
            city_rain = 0.000

        # snowing info
        try:
            js['weather'][0]['main'] == 'Snow'
            city_snow = js['snow']['3h']
        except:
            city_snow = 0    

        # append as a list of dictionaries
        city_data.append({'City':city_name, 'Country':city_country, 'Date':city_date, 
                        'Lat':city_lat, 'Lng': city_lon,'Max Temp':city_max_temp,
                        'Humidity':city_humidity,'Cloudiness': city_cload,'Wind Speed':city_wind_speed,
                        'Current Description':city_weather_description,
                        'Rain inches (last 3 hrs)':city_rain,'Snow inches (last 3 hrs)':city_snow})
    except:
        print("City not found. Skipping...")
        pass
print('-------------------------------')
print('Data Retrieval Complete        ')
print('-------------------------------')

print(len(city_data))

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | tumannyy
City not found. Skipping...
Processing Record 2 of Set 1 | makat
Processing Record 3 of Set 1 | avarua
Processing Record 4 of Set 1 | norman wells
Processing Record 5 of Set 1 | barrow
Processing Record 6 of Set 1 | pasighat
Processing Record 7 of Set 1 | georgetown
Processing Record 8 of Set 1 | magugu
Processing Record 9 of Set 1 | yar-sale
Processing Record 10 of Set 1 | saint-georges
Processing Record 11 of Set 1 | chuy
Processing Record 12 of Set 1 | butaritari
Processing Record 13 of Set 1 | longyearbyen
Processing Record 14 of Set 1 | rikitea
Processing Record 15 of Set 1 | severo-kurilsk
Processing Record 16 of Set 1 | ushuaia
Processing Record 17 of Set 1 | tabiauea
City not found. Skipping...
Processing Record 18 of Set 1 | senneterre
Processing Record 19 of Set 1 | constantine
Processing Record 20 of Set 1 | hermanus
Processing Record 21 of Set 1 | linqiong
Processing Record 2

Processing Record 38 of Set 4 | lagos
Processing Record 39 of Set 4 | lavrentiya
Processing Record 40 of Set 4 | pangoa
Processing Record 41 of Set 4 | rabo de peixe
Processing Record 42 of Set 4 | klaksvik
Processing Record 43 of Set 4 | fairbanks
Processing Record 44 of Set 4 | olafsvik
Processing Record 45 of Set 4 | yellowknife
Processing Record 46 of Set 4 | languyan
Processing Record 47 of Set 4 | lompoc
Processing Record 48 of Set 4 | fowa
Processing Record 49 of Set 4 | alyangula
Processing Record 50 of Set 4 | salinopolis
Processing Record 1 of Set 5 | swinoujscie
Processing Record 2 of Set 5 | zhuhai
Processing Record 3 of Set 5 | dianopolis
City not found. Skipping...
Processing Record 4 of Set 5 | vila velha
Processing Record 5 of Set 5 | caarapo
Processing Record 6 of Set 5 | kenai
Processing Record 7 of Set 5 | sao joao da barra
Processing Record 8 of Set 5 | poum
Processing Record 9 of Set 5 | kaitangata
Processing Record 10 of Set 5 | ukholovo
Processing Record 11 of Se

In [ ]:
# Convert the array of dictionaries to a Pandas DataFrame.
WeatherPy_Database_df = pd.DataFrame(city_data)
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]
WeatherPy_Database_df = WeatherPy_Database_df[new_column_order]
WeatherPy_Database_df.head(10)

In [ ]:
# Create the output file (CSV).
WeatherPy_Database_df = pd.DataFrame(city_data)
# Export the City_Data into a CSV.
WeatherPy_Database_df.to_csv(path_or_buf ='C:/Users/fletc/BootCamp_Class/Mod_6/World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv', index_label='City_ID')